In [54]:
data = !az role assignment list
import json
data = json.loads("\n".join(data))

In [55]:
data[0].keys()

dict_keys(['condition', 'conditionVersion', 'createdBy', 'createdOn', 'delegatedManagedIdentityResourceId', 'description', 'id', 'name', 'principalId', 'principalName', 'principalType', 'roleDefinitionId', 'roleDefinitionName', 'scope', 'type', 'updatedBy', 'updatedOn'])

In [56]:
import re

def camel_to_snake(camel_str):
    # Add an underscore before each uppercase letter, then convert the whole string to lowercase
    snake_str = re.sub(r'(?<!^)(?=[A-Z])', '_', camel_str).lower()
    return snake_str

# Test the function
test_string = "camelCaseString"
print(camel_to_snake(test_string))  # Outputs: camel_case_string

camel_case_string


In [57]:
type(""), type(1),type(None),type(0.1)

(str, int, NoneType, float)

In [58]:
from datetime import datetime

def is_datetime(string):
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%dT%H:%M:%S",
        "%Y-%m-%d",
        "%Y-%m-%dT%H:%M:%S.%fZ",
        "%Y-%m-%dT%H:%M:%S.%f%z",
        "%Y-%m-%dT%H:%M:%S%z"
    ]
    
    for fmt in formats:
        try:
            datetime.strptime(string, fmt)
            return True
        except ValueError:
            continue
    return False

# Example usage
test_strings = [
    "2023-05-23 12:34:56",
    "2023-05-23T12:34:56",
    "2023-05-23",
    "2023-05-23T12:34:56.789Z",
    "2021-05-31T19:13:02.252938+00:00",
    "invalid datetime"
]

for test_string in test_strings:
    print(f"{test_string}: {is_datetime(test_string)}")


2023-05-23 12:34:56: True
2023-05-23T12:34:56: True
2023-05-23: True
2023-05-23T12:34:56.789Z: True
2021-05-31T19:13:02.252938+00:00: True
invalid datetime: False


In [59]:
import uuid

def is_uuid(string):
    try:
        val = uuid.UUID(string, version=4)
        return True
    except ValueError:
        return False

# Example usage
test_strings = [
    "123e4567-e89b-12d3-a456-426614174000",
    "123e4567-e89b-12d3-a456-42661417400Z",
    "invalid-uuid-string"
]

for test_string in test_strings:
    print(f"{test_string}: {is_uuid(test_string)}")


123e4567-e89b-12d3-a456-426614174000: True
123e4567-e89b-12d3-a456-42661417400Z: False
invalid-uuid-string: False


In [63]:
# #[derive(Debug, Serialize, Deserialize, PartialEq)]
# pub struct RoleAssignment {
#     id: RoleAssignmentId,
#     condition: Option<Value>,
#     #[serde(rename = "conditionVersion")]
#     condition_version: Option<Value>,
#     #[serde(rename = "createdBy")]
#     created_by: String,
#     #[serde(rename = "createdOn")]
#     created_on: DateTime<Utc>,
#     #[serde(rename = "delega")]
# }


for key in data[0].keys():
    data_type = None
    if all(x[key] is None for x in data):
        data_type = "Option<Value>"
    else:
        # first pass
        lookup = {
            "String": lambda x: type(x) is str,
            "i32": lambda x: type(x) is int,
            "f32": lambda x: type(x) is float,
        }
        for k,v in lookup.items():
            if all(x[key] is None or v(x[key]) for x in data):
                if data_type is not None:
                    raise Exception(f"Complex type not supported for key {key}: was {data_type}, now {k}")
                data_type = k

        # second pass

        if data_type == "String":
            lookup = {
                "Uuid": is_uuid,
                "DateTime<Utc>": is_datetime,
            }
            for k,v in lookup.items():
                if all(x[key] is None or v(x[key]) for x in data):
                    if data_type != "String":
                        raise Exception(f"Complex type not supported for key {key}: was {data_type}, now {k}")
                    data_type = k

        optional = any(x[key] is None for x in data)
        if optional:
            data_type = f"Option<{data_type or 'Value'}>"
    
    rename = None if key == key.lower() else camel_to_snake(key)
    if key == "type":
        rename = "kind"
    if rename is not None:
        print(f"#[serde(rename = \"{key}\")]")
        print(f"{rename}: {data_type},")
    else:
        print(f"{key}: {data_type},")

condition: Option<Value>,
#[serde(rename = "conditionVersion")]
condition_version: Option<Value>,
#[serde(rename = "createdBy")]
created_by: Uuid,
#[serde(rename = "createdOn")]
created_on: DateTime<Utc>,
#[serde(rename = "delegatedManagedIdentityResourceId")]
delegated_managed_identity_resource_id: Option<Value>,
description: Option<Value>,
id: String,
name: Uuid,
#[serde(rename = "principalId")]
principal_id: Uuid,
#[serde(rename = "principalName")]
principal_name: String,
#[serde(rename = "principalType")]
principal_type: String,
#[serde(rename = "roleDefinitionId")]
role_definition_id: String,
#[serde(rename = "roleDefinitionName")]
role_definition_name: String,
scope: String,
#[serde(rename = "type")]
kind: String,
#[serde(rename = "updatedBy")]
updated_by: Uuid,
#[serde(rename = "updatedOn")]
updated_on: DateTime<Utc>,
